In [1]:
# import os
# os.chdir(r"C:\Users\Mr Cab Driver\Documents\aCreativeHub\Projekti\Speech-to-Text\LiteGRUv1\anaconda\LiteGRUv2_v2_7")
# !dir

In [2]:
import torch
torch.__version__

'2.0.1'

In [1]:
import cpuinfo

info = cpuinfo.get_cpu_info()

print("Processor:", info["brand_raw"])
print("Architecture:", info["arch"])
print("Cores:", info["count"])

Processor: 11th Gen Intel(R) Core(TM) i5-11600 @ 2.80GHz
Architecture: X86_64
Cores: 12


In [2]:
import pyopencl as cl

def get_gpu_info():
    platforms = cl.get_platforms()
    platform = platforms[0]  # Assuming you have at least one platform
    devices = platform.get_devices(device_type=cl.device_type.GPU)

    if devices:
        gpu = devices[0]  # Assuming you have at least one GPU
        return f"GPU Name: {gpu.name}\nDevice Type: {gpu.type}\nVendor: {gpu.vendor}\nOpenCL Version: {gpu.version}\nGlobal Memory: {gpu.global_mem_size / (1024 ** 3):.2f} GB\nMax Compute Units: {gpu.max_compute_units}"
    else:
        return "No GPUs found."

gpu_info = get_gpu_info()
print(gpu_info)

GPU Name: GeForce GTX 760
Device Type: 4
Vendor: NVIDIA Corporation
OpenCL Version: OpenCL 1.2 CUDA
Global Memory: 4.00 GB
Max Compute Units: 6


In [3]:
%%writefile char_to_idx.py

# Define the char_to_idx mapping
char_to_idx = {
    " ": 0, "[PAD]": 1, "[UNK]": 2, "a": 3, "b": 4, "c": 5, "d": 6, "e": 7, "f": 8,
    "g": 9, "h": 10, "i": 11, "j": 12, "k": 13, "l": 14, "m": 15, "n": 16, "o": 17,
    "p": 18, "q": 19, "r": 20, "s": 21, "t": 22, "u": 23, "v": 24, "w": 25, "x": 26,
    "y": 27, "z": 28, "ä": 29, "ü": 30, "ć": 31, "č": 32, "đ": 33, "š": 34, "ž": 35,
    "ӧ": 36, "1": 37, "2": 38, "3": 39, "4": 40, "5": 41, "6": 42, "7": 43, "8": 44,
    "9": 45, "0": 46, ".": 47
}

idx_to_char = {idx: char for char, idx in char_to_idx.items()}

Overwriting char_to_idx.py


In [6]:
%%writefile utils.py

"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model state_dict to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)




# ## FONETSKA FREKVENCIJA U TEKSTU ##

# import re
# import json
# import csv

# def load_vocab(file_path):
#     with open(file_path, 'r', encoding='utf-8') as f:
#         vocab = json.load(f)
#     return vocab

# def count_phonemes(text, vocab):
#     phoneme_counts = {phoneme: 0 for phoneme in vocab}

#     # Split the text into words and process each word
#     words = re.findall(r'\b\w+\b', text)
#     for word in words:
#         for phoneme in vocab:
#             if phoneme in word:
#                 phoneme_counts[phoneme] += 1

#     return phoneme_counts

# def calculate_average_percent(phoneme_counts):
#     total = sum(phoneme_counts.values())
#     average_percentages = {phoneme: count / total * 100 for phoneme, count in phoneme_counts.items()}
#     return average_percentages

# def write_csv(phoneme_counts, average_percentages, csv_file_path):
#     with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerow(['Phoneme', 'Count', 'Average %'])
#         for phoneme, count in phoneme_counts.items():
#             average_percent = average_percentages[phoneme]
#             writer.writerow([phoneme, count, average_percent])

# def main():
#     text_file_path = "/content/Recenice_set_B.txt"
#     vocab_file_path = "/content/vocab.json"
#     csv_file_path = "/content/"+"phoneme_freq_text_B"+".csv"

#     with open(text_file_path, 'r', encoding='utf-8') as f:
#         text = f.read()

#     vocab = load_vocab(vocab_file_path)
#     phoneme_counts = count_phonemes(text, vocab)
#     average_percentages = calculate_average_percent(phoneme_counts)

#     write_csv(phoneme_counts, average_percentages, csv_file_path)

# if __name__ == '__main__':
#     main()

# print(f".cvs FILE HAS BEEN SAVED")

Writing utils.py


In [7]:
%%writefile plot_loss_curves.py

import torch
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Dict, List

def plot_loss_curves(results: Dict[str, List[float]]):

    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}

        results is a .pkl file
    """

    train_loss = results['train_loss']
    train_acc = results['train_acc']
    test_loss = results['test_loss']
    test_acc = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot
    plt.figure(figsize=(10, 4))

    # Calculate LOSS trendlines using NumPy's polyfit
    train_trend = np.polyfit(epochs, train_loss, 3)
    test_trend = np.polyfit(epochs, test_loss, 4)

    # Calculate ACCURACY trendlines using NumPy's polyfit
    train_trend_acc = np.polyfit(epochs, train_acc, 3)
    test_trend_acc = np.polyfit(epochs, test_acc, 4)

    # Generate more points for smoother curves
    epochs_extended = np.linspace(1, epochs, 100)

    # Function to apply moving average smoothing
    def moving_average(data, window_size):
      return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

    # Set the window size for moving average smoothing (choose an appropriate value)
    window_size = 1

    # Apply moving average smoothing to the LOSS data
    smoothed_train_loss = moving_average(train_loss, window_size)
    smoothed_test_loss = moving_average(test_loss, window_size)

    # Apply moving average smoothing to the ACCURACY data
    smoothed_train_acc = moving_average(train_acc, window_size)
    smoothed_test_acc = moving_average(test_acc, window_size)

    # Adjust the epochs array to match the length of the smoothed data
    smoothed_epochs = epochs[window_size-1:]

    # Plot smoothed LOSS
    plt.subplot(1, 2, 1)
    plt.plot(smoothed_epochs, smoothed_train_loss, label='train_loss')        # no smoothing as window_size = 1!!!
    plt.plot(smoothed_epochs, smoothed_test_loss, label='test_loss')    # no smoothing as window_size = 1!!!
    plt.title('Smoothed Loss')
    plt.xlabel('Epochs')
    #plt.ylim(0.5, 1.5)
    plt.legend()

    # # Plot LOSS with trendlines
    plt.subplot(1, 2, 1)
    # plt.plot(epochs, loss, label='train_loss')
    # plt.plot(epochs, test_loss, label='test_loss')
    plt.plot(epochs, np.polyval(train_trend, epochs), '--', label='train_trend')
    plt.plot(epochs, np.polyval(test_trend, epochs), '--', label='test_trend')
    plt.title('Loss trend')
    plt.xlabel('Epochs')
    plt.legend()


    # Plot ACCURACY
    plt.subplot(1, 2, 2)
    plt.plot(epochs, smoothed_train_acc, label='train_accuracy')
    plt.plot(epochs, smoothed_test_acc, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot ACCURACY with trendlines
    plt.subplot(1, 2, 2)
    plt.plot(epochs, np.polyval(train_trend_acc, epochs), '--', label='train_trend_acc')
    plt.plot(epochs, np.polyval(test_trend_acc, epochs), '--', label='test_trend_acc')
    plt.title('Accuracy trend')
    plt.xlabel('Epochs')
    plt.ylim(0, 1.0)
    plt.legend()
    plt.show

if "__name__" == "__main__":
  plot_loss_curves()

Writing plot_loss_curves.py


In [8]:
%%writefile data_setup.py
## DATASET CLASS ##

import string
import torch
import torchaudio
import subprocess
from torch.utils.data import Dataset
import torchaudio.transforms as T
from char_to_idx import char_to_idx



# ## DEFINIRANJE ##
# ## MFCC FEATURE EXTRACTORA ##

# # Oduzima se prvi koeficijent
# n_mfcc = 13
# fs = 16000 #Hz makni ovo kasnije

# # Making a class for MFCC that does MFCC then drops the 0th coeff


class MFCCWithoutZeroth(T.MFCC):
    def __init__(self, *args, **kwargs):
        super(MFCCWithoutZeroth, self).__init__(*args, **kwargs)

    def __call__(self, waveform):
        mfcc_data = super(MFCCWithoutZeroth, self).__call__(waveform)
        mfcc_data = mfcc_data[:, 1:]
        return mfcc_data


# # Apply the custom transform
# #mfcc_data = mfcc_transform(waveform)





"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os

#from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    train_tran: str,
    test_dir: str,
    test_tran: str,
    mfcc_transform: MFCCWithoutZeroth,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    mfcc_transform: MFCC transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader).

    Example usage:
      train_dataloader, test_dataloader, = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             mfcc_transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """

  # Load audio transcriptions
  #with open(train_tran, 'r') as file:
  #    train_audio_transcriptions = [line.strip() for line in file]
  #with open(test_tran, 'r') as file:
  #    test_audio_transcriptions = [line.strip() for line in file]


  # Turn Audio sets into Dataloaders

  train_data = ASRDataset(train_dir,
                          train_tran,
                          mfcc_transform,
                          char_to_idx)

  train_dataloader = DataLoader(train_data,
                                batch_size=batch_size,
                                collate_fn=collate_fn,
                                shuffle=True,
                                num_workers=num_workers,
                                pin_memory=True,)

  test_data = ASRDataset(test_dir,
                         test_tran,
                         mfcc_transform,
                         char_to_idx)

  test_dataloader = DataLoader(test_data,
                               batch_size=batch_size,
                               collate_fn=collate_fn,
                               shuffle=False,
                               num_workers=num_workers,
                               pin_memory=True,)

  return train_dataloader, test_dataloader





## DATASET 4 ##

class ASRDataset(Dataset):
    def __init__(self, folder_path, audio_transcriptions_path, mfcc_transform, char_to_idx):
        self.folder_path = folder_path
        self.mfcc_transform = mfcc_transform
        self.char_to_idx = char_to_idx
        #self.audio_transcriptions_path = audio_transcriptions_path
        #self.wav_audio = None  # Initialize with None
        #self.audio_cache = {}  # Cache dictionary

        audio_filenames = sorted(os.listdir(self.folder_path))
        self.audio_file_paths = [os.path.join(self.folder_path, filename) for filename in sorted(audio_filenames)]


        # # Load audio filenames and transcriptions into memory
        # with open(audio_filenames_path, 'r') as file:
        #     self.audio_filenames = [line.strip() for line in file]

        with open(audio_transcriptions_path, 'r', encoding="utf-8") as file:
            self.audio_transcriptions = [line.strip() for line in file]

        # Check if the lengths match
        # assert len(self.audio_filenames) == len(self.audio_transcriptions), "Filenames and transcriptions do not match"


    def __len__(self):
        return len(self.audio_transcriptions)


    def preprocess_transcription(self, trans):
        # Convert to lowercase
        trans = trans.lower()

        #print(f"class after trans: {trans}")

        # Remove punctuation marks

        #trans = ' '.join('[UNK]' if word in set_of_special_symbols else word for word in trans.split())
        #trans = ' '.join('[UNK]' if word in string.punctuation else word for word in trans.split())
        #trans = ' '.join(char_to_idx.get(word, '[UNK]') for word in trans.split())

        trans = ''.join(char for char in trans if char not in string.punctuation)



        #unk_index = char_to_idx.get('[UNK]', -1)  # Get the index for the [UNK] token, default to -1 if not found
        #trans = ' '.join(str(char_to_idx.get(word, unk_index)) for word in trans.split())
        #print(f"class before trans: {trans}")


        #trans = ''.join(char for char in trans if char not in string.punctuation + '“”’‘—-')
        #print(trans)

        # Process other special symbols if needed

        return trans


    def __getitem__(self, idx):
        # Get the corresponding transcription
        audio_transcription = self.audio_transcriptions[idx]


        # Construct the full audio file path based on the index
        audio_filename = sorted(os.listdir(self.folder_path))[idx]
        audio_file_path = os.path.join(self.folder_path, audio_filename)

        # Load audio using torchaudio
        mp3_audio, orig_freq = torchaudio.load(audio_file_path)

        # Resample audio to 16000 Hz
        resampler = torchaudio.transforms.Resample(orig_freq=orig_freq, new_freq=16000)
        wav_audio = resampler(mp3_audio)




        #self.wav_audio = wav_audio
        #print(f"class self.wav_audio: {wav_audio}")

        # Apply MFCC transformation
        mfcc = self.mfcc_transform(wav_audio).permute(0, 2, 1)

        # Convert transcription to target labels using char_to_idx
        processed_transcription = self.preprocess_transcription(audio_transcription)
        target_labels = [self.char_to_idx[c] if c in self.char_to_idx else self.char_to_idx["[UNK]"] for c in processed_transcription]

        return mfcc, target_labels



## PRETVARANJE MFCC U 1D +PAD ##

from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Unzip the batch into a list of MFCC sequences and a list of transcriptions
    mfccs, transcriptions = zip(*batch)


    # print(f"col transcriptions: {transcriptions}")

    # Find the maximum sequence length along both dimensions
    max_time_steps = max(mfcc.shape[1] for mfcc in mfccs)
    #num_mfcc_coeffs = mfccs[0].shape[2]  # Assuming all tensors have the same number of MFCC coefficients

    #print(f"col max length here: {max_time_steps}")
    #print(f"col mfcc coeffs: {num_mfcc_coeffs}")

    # for item in mfccs:
    #     print(f"col item.shape {item.shape}")

    # Pad the transposed MFCC sequences to have the same length
    mfccs = [torch.nn.functional.pad(mfcc, (0, 0, 0, max_time_steps - mfcc.shape[1])) for mfcc in mfccs]
    #print(f"col mfccs padded len {len(mfccs)}")

    # for item in mfccs:
    #     print(f"col item.shape padded {item.shape}")

    # Stack the padded MFCC sequences into a batched tensor
    mfccs = torch.stack(mfccs)

    # Calculate the maximum length of transcriptions in the batch
    max_transcription_length = max(len(trans) for trans in transcriptions)
    # print(f"col max_transcription_length {max_transcription_length}")

    # Pad the transcriptions to have the same length using [PAD] index
    padded_transcriptions = [trans + [char_to_idx['[PAD]']] * (max_transcription_length - len(trans)) for trans in transcriptions]
    targets = torch.tensor(padded_transcriptions)


    # These targets are unpadded originals!!!!
    # targets = [torch.tensor(target) for mfcc, target in batch]

    # Return the batched MFCC sequences and transcriptions
    return mfccs, targets

Writing data_setup.py


In [9]:
%%writefile model_builder.py
## DEFINIRANJE MODELA ##

import torch
import torch.nn as nn
import torch.nn.functional as F

class LiteGRUv2(nn.Module):

  """Creates a Lightweight LiteGRUv2 model for ASR.

  Args:
  num_classes: An integer indicating number of classes
  in_channels: An integer indicating number of input features! # Number of MFCC coefficients, should be dynamic tho
  hidden_units: An integer indicating number of hidden units between layers.
  dropout: A float indicating the dropout rate (0.1 or lower is ok 12.8.2023.).
  """

  def __init__(self, num_classes, in_channels, hidden_units, dropout):
      super(LiteGRUv2, self).__init__()

      self.conv1d = nn.Conv1d(in_channels=in_channels, out_channels=hidden_units, kernel_size=5, padding=2)  # Equivalent to padding='same'
      self.batchnorm1 = nn.BatchNorm1d(hidden_units)
      self.birnn = nn.GRU(input_size=hidden_units, hidden_size=hidden_units, bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(p=dropout)
      self.batchnorm2 = nn.BatchNorm1d(hidden_units * 2)  # 128 (hidden size) * 2 (bidirectional)
      self.fc = nn.Linear(hidden_units * 2, num_classes)
      #self.softmax = nn.Softmax(dim=2)  # Apply softmax over the time dimension

  def forward(self, x):
      # print(f"fw x.shape entering: {x.shape}")
      x = self.conv1d(x)
      # print(f"fw conv1d.shape: {x.shape}")
      x = self.batchnorm1(x)
      # print(f"fw batchnorm1.shape: {x.shape}")
      x = x.permute(0, 2, 1) # Expected shape for GRU input: (batch_size, sequence_length, input_size)
      x, _ = self.birnn(x)
      # print(f"fw birnn(GRU).shape: {x.shape}")
      x = self.dropout(x)
      # print(f"fw dropout.shape: {x.shape}")
      x = x.permute(0, 2, 1)
      x = self.batchnorm2(x)
      # print(f"fw batchnorm2.shape: {x.shape}")
      x = x.permute(0, 2, 1)
      x = self.fc(x)
      # print(f"fw fc.shape: {x.shape}")
      #x = self.softmax(x)
      # print(f"fw softmax.shape: {x.shape}")
      return x

# Create an instance of the LiteGRUv2
# dropout = 0.1 # ne flukturira ko 0.2!
# num_classes = 30  # Change this to your number of classes
# in_channels = 12  # Number of MFCC coefficients, should be dynamic tho

Writing model_builder.py


In [10]:
%%writefile engine.py

"""
Contains functions for training and testing a PyTorch model.
"""

#import sys
import torch
import torch.optim as optim
from char_to_idx import idx_to_char

#from train import HIDDEN_UNITS
#from train import flush_cuda
import torch.nn.utils.rnn as rnn_utils
# import torch.nn.functional as F

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

from timeit import default_timer as timer

# print every which freq:
freq = 20



def calculate_ctc_accuracy(logits, target_lengths, transcriptions, device):

    # print(f"ctcacc logits.shape: {logits.shape}")
    # print(f"ctcacc target_lengths.shape: {target_lengths.shape}")
    # print(f"ctcacc target_lengths: {target_lengths}")
    # print(f"ctcacc transcriptions.shape: {transcriptions.shape}")
    # print(f"ctcacc transcriptions: {transcriptions}")

    # Convert logits to predicted labels using argmax
    predicted_labels = torch.argmax(logits, dim=2)

    total_chars = 0
    correct_chars = 0

    # print(f"ctcacc predicted labels from logits: {predicted_labels}")
    # print(f"ctcacc predicted labels length: {len(predicted_labels)}")
    # print(f"ctcacc predicted labels shape: {predicted_labels.shape}")


    for i in range(predicted_labels.shape[1]):


        predicted_seq = predicted_labels[:target_lengths[i], i].to(device)  # Trim padding from predicted sequence
        reference_seq = transcriptions[i][:target_lengths[i]].to(device) # Trim padding from reference sequence

        total_chars += len(reference_seq)
        correct_chars += torch.sum(predicted_seq == reference_seq).item()


        accuracy = correct_chars / total_chars

    return accuracy


def train_step(model: torch.nn.Module,
               train_dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               num_classes: int,
               device: torch.device) -> Tuple[float, float]:

    model.train()  # Set the model to training mode

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    total_train_batches = len(train_dataloader)

    for batch_idx, (batch_mfccs, batch_targets) in enumerate(train_dataloader):

        if batch_idx == 0:
            print("Working on the first batch...")
        elif batch_idx == total_train_batches // 2:
            print("Working on the middle batch...")
        elif batch_idx == total_train_batches - 1:
            print("Working on the last batch...")


        batch_mfccs = batch_mfccs.squeeze(1).permute(0, 2, 1).to(device)
        # print(f"tr batch_mfccs.shape: {batch_mfccs.shape}, batch_mfccs: {batch_mfccs}")

        batch_targets = [target.to(device) for target in batch_targets]
        # print(f"tr batch_targets: {batch_targets}")

        target_lengths = torch.tensor([len(target) - (target == 1).sum().item() for target in batch_targets], dtype=torch.long)
        # print(f"tr target_lengths: {target_lengths}")
        # print(f"tr target_lengths.size: {target_lengths.size}")


        max_sequence_length = max(target_lengths)
        # print(f"tr max_sequence_length: {max_sequence_length}")



        outputs = model(batch_mfccs) ########################################################################## UPALI ME #######################################
        #outputs = torch.randn(batch_mfccs.size(0), batch_mfccs.size(2), num_classes).to(device) ####### I OVO UGASITI ONDA



        # print(f"tr outputs.shape: {outputs.shape}, outputs: {outputs}")
        outputs = torch.permute(outputs, (1, 0, 2))
        # print(f"tr batch_mfccs.size(2): {batch_mfccs.size(2)}")
        # print(f"tr outputs.premuted.shape: {outputs.shape} || CTCLoss expects: [T max time length, N batch_size, C num_classes]")




        outputs = outputs.view(batch_mfccs.size(2), len(batch_mfccs), num_classes)
        #outputs = outputs.reshape(batch_mfccs.size(0), max_sequence_length, num_classes)
        #print(f"tr outputs shape view: {outputs.shape}")

        #outputs = F.log_softmax(outputs, dim=2)

        input_lengths = torch.tensor([outputs.shape[0] for _ in batch_mfccs])
        #input_lengths_list = input_lengths.tolist()

        # print(f"tr input_lengths: {input_lengths}")
        # print(f"tr input_lengths.shape: {input_lengths.shape}")
        # print(f"tr input_lengths_list: {input_lengths_list}")


        # Initialize an empty tensor to hold the padded targets
        targets = torch.zeros(len(batch_targets), max_sequence_length, dtype=torch.long)
        # print(f"tr targets raw: {targets}")
        # print(f"tr targets raw.shape: {targets.shape}")

        target_lengths = torch.tensor([len(target) - (target == 1).sum().item() for target in batch_targets], dtype=torch.long)
        # print(f"tr target_lengths: {target_lengths}")
        # print(f"tr target_lengths.shape: {target_lengths.shape}")

        for i, target in enumerate(batch_targets):
            targets[i, :len(target)] = target

        # print(f"tr targets for batch_targets: {targets}")
        # print(f"tr targets.shape for batch_targets: {targets.shape}")



        # Create a PackedSequence object
        packed_sequence = rnn_utils.pack_padded_sequence(targets, target_lengths, batch_first=True, enforce_sorted=False)

        # Access the depadded tensor and lengths
        depadded_targets, depadded_lengths = rnn_utils.pad_packed_sequence(packed_sequence, batch_first=True)

        # print("tr depadded_targets:")
        # print(depadded_targets)

        # print("tr depadded lengths:")
        # print(depadded_lengths)



        # depadded_targets = torch.zeros(len(batch_targets), max(target_lengths), dtype=torch.long)

        # print(f"tr depadded_targets: {depadded_targets}")
        # print(f"tr depadded_targets.shape: {depadded_targets.shape}")

        # for i, target in enumerate(batch_targets):
        #     depadded_targets[i, :target_lengths[i]] = target[target != 1]  # Exclude padding token (index 1)

        # print(f"tr depadded_targets for: {depadded_targets}")
        # print(f"tr depadded_targets.shape for : {depadded_targets.shape}")






        # Train CTC accuracy in this batch
        batch_acc = calculate_ctc_accuracy(outputs, target_lengths, depadded_targets, device)

        # print(f"tr targets: {targets}")
        # print(f"tr targets.shape: {targets.shape}")

        # Loss function
        loss = loss_fn(outputs, depadded_targets.float(), input_lengths, target_lengths)
        # print(f"tr loss: {loss}, batch: {batch_idx}")

        # Boiler plate (Zero the gradients, Backpropagation, Optimizer step)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        # # Calculate gradient norms without accumulating gradients
        # with torch.autograd.no_grad():
        #     for name, param in model.named_parameters():
        #         if param.grad is not None:
        #             print(f"tr Gradient norm for {name}: {torch.norm(param.grad).item()}")


        # Train total Loss and Acc
        train_loss += loss.item()
        train_acc += batch_acc

    # Average loss and acc per batch
    train_loss = train_loss / len(train_dataloader)
    train_acc = train_acc / len(train_dataloader)
    return train_loss, train_acc
    #print(f"total_train_loss: {total_train_loss:.4f}")




    # Save the model state_dict()


    model_checkpoint = "CHECKPOINT_LiteGRUv2.6_temp.pth"

    model_save_path = "/content/drive/MyDrive/pytorch/LiteGRUv1/cv-corpus-14.0-delta-2023-06-23/en/models/" + model_checkpoint


    torch.save(obj=model.state_dict(),
               f=model_save_path)
    print(f"[SAVED] Saving CHECKPOINT model state_dict to: {model_save_path}")



def test_step(model: torch.nn.Module,
              test_dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              num_classes: int,
              device: torch.device) -> Tuple[float, float]:

    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():

        # Loop through TEST DataLoader batches

        #test_iterator = tqdm(test_dataloader, desc='Testing', leave=False)


        total_test_batches = len(test_dataloader)

        for batch_idx, (batch_mfccs, batch_targets) in enumerate(test_dataloader):





            batch_mfccs = batch_mfccs.squeeze(1).permute(0, 2, 1).to(device)
            # print(f"te batch_mfccs.shape: {batch_mfccs.shape}, batch_mfccs: {batch_mfccs}")

            batch_targets = [target.to(device) for target in batch_targets]
            # print(f"te batch_targets: {batch_targets}")

            target_lengths = torch.tensor([len(target) - (target == 1).sum().item() for target in batch_targets], dtype=torch.long)
            # print(f"te target_lengths: {target_lengths}")
            # print(f"te target_lengths.size: {target_lengths.size}")

            max_sequence_length = max(target_lengths)
            # print(f"te max_sequence_length: {max_sequence_length}")


            outputs = model(batch_mfccs) ########################################################################## UPALI ME #######################################
            #outputs = torch.randn(batch_mfccs.size(0), batch_mfccs.size(2), num_classes).to(device) ####### I OVO UGASITI ONDA


            # print(f"tr outputs.shape: {outputs.shape}, outputs: {outputs}")
            outputs = torch.permute(outputs, (1, 0, 2))
            # print(f"te batch_mfccs.size(2): {batch_mfccs.size(2)}")
            # print(f"te outputs.premuted.shape: {outputs.shape} || CTCLoss expects: [T max time length, N batch_size, C num_classes]")

            outputs = outputs.view(batch_mfccs.size(2), len(batch_mfccs), num_classes)
            #outputs = outputs.reshape(batch_mfccs.size(0), max_sequence_length, num_classes)
            #print(f"te outputs shape view: {outputs.shape}")

            #outputs = F.log_softmax(outputs, dim=2)

            input_lengths = torch.tensor([outputs.shape[0] for _ in batch_mfccs])
            #input_lengths_list = input_lengths.tolist()

            # print(f"te input_lengths: {input_lengths}")
            # print(f"te input_lengths.shape: {input_lengths.shape}")
            # print(f"te input_lengths_list: {input_lengths_list}")

            # Initialize an empty tensor to hold the padded targets
            targets = torch.zeros(len(batch_targets), max_sequence_length, dtype=torch.long)
            # print(f"te targets raw: {targets}")
            # print(f"te targets raw.shape: {targets.shape}")

            target_lengths = torch.tensor([len(target) - (target == 1).sum().item() for target in batch_targets], dtype=torch.long)
            # print(f"te target_lengths: {target_lengths}")
            # print(f"te target_lengths.shape: {target_lengths.shape}")

            for i, target in enumerate(batch_targets):
                targets[i, :len(target)] = target

            # print(f"te targets for batch_targets: {targets}")
            # print(f"te targets.shape for batch_targets: {targets.shape}")

            # Create a PackedSequence object
            packed_sequence = rnn_utils.pack_padded_sequence(targets, target_lengths, batch_first=True, enforce_sorted=False)

            # Access the depadded tensor and lengths
            depadded_targets, depadded_lengths = rnn_utils.pad_packed_sequence(packed_sequence, batch_first=True)

            # print("te depadded_targets:")
            # print(depadded_targets)

            # print("te depadded lengths:")
            # print(depadded_lengths)

            # Train CTC accuracy in this batch
            batch_acc = calculate_ctc_accuracy(outputs, target_lengths, depadded_targets, device)

            # print(f"te targets: {targets}")
            # print(f"te targets.shape: {targets.shape}")

            # Loss function
            loss = loss_fn(outputs, depadded_targets.float(), input_lengths, target_lengths)
            # print(f"te loss: {loss}, batch: {batch_idx}")


            # ###########################################################################################################

            # # print(f"te batch_idx: {batch_idx}")

            # # Send data to target device
            # batch_mfccs = batch_mfccs.squeeze(1).permute(0, 2, 1).to(device)
            # batch_targets = [target.to(device) for target in batch_targets]


            # #if len(batch_mfccs) < batch_idx:
            # #    break




            # # Forward pass

            # # print(f"te batch_mfccs: {batch_mfccs}")
            # # print(f"te batch_targets: {batch_targets}")

            # outputs = model(batch_mfccs)
            # # print(f"te outputs from model(batch_mfccs): {outputs}")
            # # print(f"te outputs.shape straight outta compton: {outputs.shape}")

            # outputs = torch.permute(outputs, (1, 0, 2))
            # # print(f"te batch_mfccs.size(2): {batch_mfccs.size(2)}")
            # # print(f"te outputs.premuted.shape: {outputs.shape} || CTCLoss expects: [T max time length, N batch_size, C num_classes]")


            # outputs = outputs.view(batch_mfccs.size(2), len(batch_mfccs), num_classes)
            # # print(f"te outputs shape view: {outputs.shape}")


            # #outputs = F.log_softmax(outputs, dim=2)

            # # input_lengths = torch.tensor([mfcc.shape[1] for mfcc in batch_mfccs])
            # # input_lengths_list = input_lengths.tolist()

            # #input_lengths = torch.tensor([mfcc.shape[1] for mfcc in batch_mfccs])

            # #target_lengths = torch.tensor([len(trans) for trans in batch_targets], dtype=torch.long)


            # #input_lengths = torch.tensor([mfcc.shape[1] for mfcc in batch_mfccs])
            # input_lengths = torch.tensor([outputs.shape[0] for _ in batch_mfccs])

            # input_lengths_list = input_lengths.tolist()

            # # Calculate target lengths excluding [PAD] tokens (index 1)
            # target_lengths = torch.tensor([len(target) - (target == 1).sum().item() for target in batch_targets], dtype=torch.long)

            # #target_lengths = torch.tensor([len(trans) for trans in batch_targets], dtype=torch.long)
            # target_lengths_list = target_lengths.tolist()


            # # Calculate the maximum sequence length
            # max_sequence_length = max(input_lengths)


            # # # Initialize an empty tensor to hold the padded targets
            # # targets = torch.zeros(len(batch_targets), max_sequence_length, dtype=torch.long)

            # # for i, target in enumerate(batch_targets):
            # #     targets[i, :target.size(0)] = target  # Copy values into the targets tensor



            # targets = torch.zeros(len(batch_targets), max_sequence_length, dtype=torch.long)

            # for i, target in enumerate(batch_targets):
            #     # Filter out [PAD] token (index 1)
            #     target_filtered = target[target != 1]
            #     targets[i, :target_filtered.size(0)] = target_filtered  # Copy values into the targets tensor

            # # Fill in the padded target sequences
            # for i, target_seq in enumerate(targets):  # Use the filtered targets instead
            #     target_seq = target_seq[:target_lengths[i]]
            #     targets[i, :target_lengths[i]] = target_seq



            # # Test CTC accuracy in this batch
            # batch_acc = calculate_ctc_accuracy(outputs, target_lengths, targets, device)

            # # epsilon = 1e-8
            # # outputs = torch.log(outputs + epsilon)
            # #outputs = torch.log(outputs) ## apsolutno vitalno da ctcloss ne bude negativan bruh
            # # print(f"te log outputs: {outputs}")


            # # Fill in the padded target sequences
            # # for i, target_seq in enumerate(batch_targets):
            # #     targets[i, :target_lengths[i]] = target_seq


            # # Loss function
            # loss = loss_fn(outputs, targets.float(), input_lengths, target_lengths)
            # print(f"te loss: {loss}, batch: {batch_idx}")
            ###################################################################################


            # Test total Loss
            test_loss += loss.item()

            # Test total Accuracy
            test_acc += batch_acc

            # Print out status:
            #tqdm.write(f" Batch [{batch_idx+1}/{len(test_dataloader)}], test Loss: {loss.item():.4f}, test Accuracy: {batch_acc:.4f}")
            #sys.stdout.flush()

            #test_iterator.set_description(f'Testing - Loss: {loss.item():.4f}, Acc: {batch_acc:.4f}')

            predicted_labels = torch.argmax(outputs, dim=2)
            #print(f"te predicted_labels: {predicted_labels}, batch: {batch_idx}")

            predicted_seq = predicted_labels[:target_lengths[i], i].to(device)  # Trim padding from predicted sequence
            #print(f"te predicted_seq: {predicted_seq}, batch: {batch_idx}")


            reference_seq = targets[i][:target_lengths[i]].to(device) # Trim padding from reference sequence
            #print(f"te reference_seq: {reference_seq}, batch: {batch_idx}")

            if batch_idx == 0:
                print("Working on the first batch...")

                # print one example:
                transcription = [idx_to_char[idx.item()] for idx in predicted_seq]
                transcription = ''.join(transcription)
                c_transcription = [idx_to_char[idx.item()] for idx in reference_seq]
                c_transcription = ''.join(c_transcription)

                # Print the transcription
                print(f"testfirst correct Transcription: \n{c_transcription}")
                print(f"testfirst Transcription: \n{transcription}")


            elif batch_idx == total_test_batches // 2:
                print("Working on the middle batch...")

                # print one example:
                transcription = [idx_to_char[idx.item()] for idx in predicted_seq]
                transcription = ''.join(transcription)
                c_transcription = [idx_to_char[idx.item()] for idx in reference_seq]
                c_transcription = ''.join(c_transcription)

                # Print the transcription
                print(f"testmid correct Transcription: \n{c_transcription}")
                print(f"testmid Transcription: \n{transcription}")


            elif batch_idx == total_test_batches - 1:
                print("Working on the last batch...")

                # print one example:
                transcription = [idx_to_char[idx.item()] for idx in predicted_seq]
                transcription = ''.join(transcription)
                c_transcription = [idx_to_char[idx.item()] for idx in reference_seq]
                c_transcription = ''.join(c_transcription)

                # Print the transcription
                print(f"testlast correct Transcription: \n{c_transcription}")
                print(f"testlast Transcription: \n{transcription}")



            # counter = 0

            # # PER BATCH? so every 20th batch is now:
            # for i in range(predicted_labels.shape[1]):

            #     counter += 1
            #     if counter % 20 == 0:

            #         #transcription = [idx_to_char[idx.item()] for idx in predicted_seq[:, i]]
            #         transcription = [idx_to_char[idx.item()] for idx in predicted_seq]
            #         transcription = ''.join(transcription)

            #         #c_transcription = [idx_to_char[idx.item()] for idx in reference_seq[:, i]]
            #         c_transcription = [idx_to_char[idx.item()] for idx in reference_seq]
            #         c_transcription = ''.join(c_transcription)

            #         # Print the transcription
            #         print(f"test100 correct Transcription: \n{c_transcription}")
            #         print(f"test100 Transcription: \n{transcription}")


        # Average loss and acc per batch
        test_loss = test_loss / len(test_dataloader)
        test_acc = test_acc / len(test_dataloader)
        # print(f"counter {counter}")
        return test_loss, test_acc




## COMBINATION OF TRAIN AND TEST:

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          num_classes: int,
          device: torch.device,) -> Dict[str, List]:

    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary with extra info
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": [],
               "duration_s": [],
               "model": str(model),
               "train_dataloader": str(train_dataloader),
               "test_dataloader": str(test_dataloader),
               "optimizer": str(optimizer),
               "loss_fn": str(loss_fn),
               "epochs": str(epochs),
               "device": str(device)
    }

    # Start the timer
    start_time = timer()
    #print(f"started timer[{timer()}s]")

    # Make sure model on target device
    model.to(device)
    #print(f"model on device[{timer()}s]")






    ### EPOCH LOOPING IS HERE ###

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs), desc="Epochs"):
        #print(f"epochs: {epochs}")

        end_time = timer() - start_time
        print(f"[{end_time:.2f}s] about to do train_step, epoch {epoch+1}")
        train_loss, train_acc = train_step(model=model,
                                          train_dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          num_classes=num_classes,
                                          device=device)

        end_time = timer() - end_time
        print(f"[{end_time:.2f}s] about to do test_step, epoch {epoch+1}")
        test_loss, test_acc = test_step(model=model,
                                        test_dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        num_classes=num_classes,
                                        device=device)

        # Print out what's happening
        print(
                      f" Epoch: {epoch+1:02d} | "
                      f"train_loss: {train_loss:.3f} | "
                      f"train_acc: {train_acc:.3f} | "
                      f"test_loss: {test_loss:.3f} | "
                      f"test_acc: {test_acc:.3f}",
        )

        #sys.stdout.flush()

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        # Empty GPU cache if batch size or len(train_dataloader) is larger than a threshold
        #if device == "cuda" and (len(train_dataloader)*HIDDEN_UNITS > flush_cuda):
        #    torch.cuda.empty_cache()

    end_time = timer()
    results["duration_s"].append(end_time - start_time)

    # Return the filled results at the end of the epochs
    return results

Writing engine.py


In [11]:
%%writefile prediction.py

## PREDICTION

import torch
import torchaudio
import argparse
import random

from pyctcdecode import build_ctcdecoder

import torch.nn.functional as F

import model_builder
from char_to_idx import idx_to_char, char_to_idx
from typing import Tuple

import torchaudio.transforms as T
from data_setup import MFCCWithoutZeroth, ASRDataset

# # Creating a parser
# parser = argparse.ArgumentParser()


model_path = "/content/drive/MyDrive/pytorch/LiteGRUv1/cv-corpus-14.0-delta-2023-06-23/en/models/"
model_pth_name = "LiteGRUv2_model_test_12.pth"


# # Pick an audio path
# parser.add_argument("--audio",
#                     default= "/content/drive/MyDrive/pytorch/LiteGRUv1/cv-corpus-14.0-delta-2023-06-23/en/test",
#                     type=str,
#                     help="target audio to transcribe!")
# # Pick a model path
# parser.add_argument("--model_path",
#                     default= model_path + model_pth_name,
#                     type=str,
#                     help="provide folder and name of saved model to be used for prediction/transcription")

# args = parser.parse_args()


# # Get audio and model paths
# AUDIO_PATH = args.audio
# print(f"[INFO] Transcribing: {AUDIO_PATH}")
# MOD_PATH = args.model_path
# print(f"[INFO] with model: {model_pth_name}")



AUDIO_PATH = "/content/drive/MyDrive/pytorch/LiteGRUv1/cv-corpus-14.0-delta-2023-06-23/en/test"
print(f"[INFO] Transcribing: {AUDIO_PATH}")
MOD_PATH = "/content/drive/MyDrive/pytorch/LiteGRUv1/cv-corpus-14.0-delta-2023-06-23/en/models/"
print(f"[INFO] with model: {model_pth_name}")

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"



### LOAD MODEL ###

def load_model(filepath=MOD_PATH):
  model=model_builder.LiteGRUv2(num_classes = 48,
                                in_channels = n_mfccs-1,
                                hidden_units = 256,
                                dropout = 0.1).to(device)

  print(f"[INFO] Loading model...")

  # Load the model's state dictionary and map tensors to CPU
  model.load_state_dict(torch.load(filepath, map_location=torch.device('cpu')))

  return model








### TRANSCRIBE TEXT ###

def transcribe(model: torch.nn.Module,
               audio_data: int,
               device: torch.device) -> str:

    # Put model in eval mode
    model.eval()

    model = model.to(device)

    # Turn on inference context manager
    with torch.inference_mode():

        print(f"pred audio_data.shape raw: {audio_data.shape}")

        audio_data = audio_data.squeeze(1).permute(0, 2, 1).to(device)
        print(f"pred audio_data.shape after: {audio_data.shape}")

        outputs = model(audio_data)

        print(f"pred outputs.shape: {outputs.shape}")
        print(f"pred output logits: {outputs}")

        outputs = F.log_softmax(outputs, dim=2)
        print(f"pred log_softmax outputs: {outputs}")

        predicted_labels = torch.argmax(outputs, dim=2)
        print(f"pred predicted_labels.shape: {predicted_labels.shape}")
        print(f"pred predicted_labels after argmax: {predicted_labels}")

        #transcription = [idx_to_char[idx] for idx in predicted_labels]
        transcription = [idx_to_char[idx.item()] for idx in predicted_labels[0]]  # Take the first sequence

        transcription = ''.join(transcription)

        # Print the transcription
        print("pred Transcription:", transcription)

    return transcription




n_mfccs = 13

model = load_model(MOD_PATH+model_pth_name)


mfcc_transform = MFCCWithoutZeroth(sample_rate = 16000,
                                   n_mfcc = n_mfccs,
                                   melkwargs = {"n_fft": 400,
                                              "hop_length": 160,
                                              "n_mels": 40}
)


test_dir = "/content/drive/MyDrive/pytorch/LiteGRUv1/cv-corpus-14.0-delta-2023-06-23/en/test"
test_tran = "/content/drive/MyDrive/pytorch/LiteGRUv1/cv-corpus-14.0-delta-2023-06-23/en/test_trans_validated.txt"


# Create an instance of your dataset
trans_data = ASRDataset(test_dir,
                        test_tran,
                        mfcc_transform,
                        char_to_idx)

num_samples = len(trans_data)

# Generate a random index within the range of available samples
#random_idx = random.randint(0, num_samples - 1)
random_idx = 619

# Fetch the data sample using the random index
sample_mfcc, sample_label = trans_data[random_idx]


transcrip = transcribe(model, sample_mfcc, device)

print(f"TRANSCRIPTION: {transcrip}")
print(f"CORRECT: {sample_label}")

# if __name__ == "__main__":
#   transcribe()


Writing prediction.py


In [12]:
%%writefile train.py

"""
Trains a PyTorch ASR LiteGRUv2 model. New build or do additional training.
"""
import torch.nn as nn
import sys
import re
import os
import argparse
import pickle
import torch
import data_setup, engine, model_builder, utils
import torchaudio.transforms as T
from data_setup import MFCCWithoutZeroth, create_dataloaders
import char_to_idx


NUM_CLASSES = len(char_to_idx.char_to_idx)

parser = argparse.ArgumentParser(description="Get hyperparameters")


parser.add_argument("--num_epochs", default=3, type=int, help="the number of epochs to train for")
parser.add_argument("--batch_size", default=64, type=int, help="number of samples per batch")
parser.add_argument("--hidden_units", default=512, type=int, help="number of hidden units in hidden layers")
parser.add_argument("--learning_rate", default=0.001, type=float, help="the learning rate for the model")

# Add the option for "new" or "old" model
parser.add_argument("--model_option", choices=["new", "old"], default="old", help="Train a new or old model")
parser.add_argument("--old_model_path", type=str, help="path to the old model checkpoint")

parser.add_argument("--n_mfccs", default=13, type=int, help="number of MFCC coefficients")
parser.add_argument("--dropout", default=0.1, type=float, help="dropout rate")
parser.add_argument("--train_dir", default=r"C:\Users\Mr Cab Driver\Documents\aCreativeHub\Projekti\Speech-to-Text\LiteGRUv1\cv-corpus-14.0-delta-2023-06-23-en\cv-corpus-14.0-delta-2023-06-23\en\clips_4k_set_only_validated\train", type=str, help="the train data directory")
parser.add_argument("--test_dir", default=r"C:\Users\Mr Cab Driver\Documents\aCreativeHub\Projekti\Speech-to-Text\LiteGRUv1\cv-corpus-14.0-delta-2023-06-23-en\cv-corpus-14.0-delta-2023-06-23\en\clips_4k_set_only_validated\test", type=str, help="the test data directory")
parser.add_argument("--train_tran", default=r"C:\Users\Mr Cab Driver\Documents\aCreativeHub\Projekti\Speech-to-Text\LiteGRUv1\cv-corpus-14.0-delta-2023-06-23-en\cv-corpus-14.0-delta-2023-06-23\en\clips_4k_set_only_validated\train_trans_validated.txt", type=str, help="the train data directory")
parser.add_argument("--test_tran", default=r"C:\Users\Mr Cab Driver\Documents\aCreativeHub\Projekti\Speech-to-Text\LiteGRUv1\cv-corpus-14.0-delta-2023-06-23-en\cv-corpus-14.0-delta-2023-06-23\en\clips_4k_set_only_validated\test_trans_validated.txt", type=str, help="the test data directory")




args = parser.parse_args()

if args.model_option == "new":
  print(f"[INFO] Training a NEW model from scratch!")
else:
  print(f"[INFO] Training an OLD model from old_model_path!")


# Setup hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
N_MFCCS = args.n_mfccs
DROPOUT = args.dropout
TRAIN_DIR = args.train_dir
TEST_DIR = args.test_dir
TRAIN_TRAN = args.train_tran
TEST_TRAN = args.test_tran
OLD_MODEL_PATH = args.old_model_path



# flush_cuda = 65537 # 64 batch * 1024 hidden +1

print(f"[INFO] Training a model for {NUM_EPOCHS} epochs with a batch size {BATCH_SIZE} using {HIDDEN_UNITS} hidden units and a learning rate of {LEARNING_RATE}\n")

# Setup directories
train_dir = args.train_dir
test_dir = args.test_dir
train_tran = args.train_tran
test_tran = args.test_tran

n_train = sum(1 for filename in os.listdir(train_dir) if filename.lower().endswith(('.mp3', '.wav')))
n_test = sum(1 for filename in os.listdir(test_dir) if filename.lower().endswith(('.mp3', '.wav')))



print(f"[INFO] Training data: {n_train} audio files \n{train_dir}")
print(f"[INFO] Testing data: {n_test} audio files \n{test_dir}")

print(f"[INFO] Training data transcriptions: \n{train_tran}")
print(f"[INFO] Testing data transcriptions: \n{test_tran}\n")

print(f"[INFO] char_to_idx contains: {NUM_CLASSES} classes\n")

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] device set to: {device}\n")



# Create the MFCC transformer object
mfcc_transform = MFCCWithoutZeroth(sample_rate = 16000,
                                   n_mfcc = N_MFCCS,
                                   melkwargs = {"n_fft": 400,
                                              "hop_length": 160,
                                              "n_mels": 40}
)

print(f"[INFO] mfcc_transform active: {mfcc_transform}\n")
print(f"[INFO] n_mfccs output: {N_MFCCS-1}\n")

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader = data_setup.create_dataloaders(
                                      train_dir=train_dir,
                                      train_tran=train_tran,
                                      test_dir=test_dir,
                                      test_tran=test_tran,
                                      mfcc_transform=mfcc_transform,
                                      batch_size=BATCH_SIZE
  )

print(f"[INFO] train_dataloader active: {train_dataloader}\n")
print(f"[INFO] test_dataloader active: {test_dataloader}\n")

print(f"[INFO] preparing model: {test_dataloader}\n")


def load_model_with_params(model_pth_path):
    # Derive the .pkl file path from the .pth file path
    model_pkl_path = model_pth_path[:-3] + 'pkl'

    # Load the state dictionary from the .pth file
    model_state_dict = torch.load(model_pth_path, map_location=torch.device('cpu'))

    # Load additional model information from the .pkl file
    with open(model_pkl_path, 'rb') as f:
        model_info = pickle.load(f)

    # Extract model architecture details
    model_architecture = model_info['model']




    # Use regular expressions to find and extract the necessary values
    in_channels_match = re.search(r'Conv1d\((\d+)', model_architecture)
    if in_channels_match:
        in_channels = int(in_channels_match.group(1))
    else:
        raise ValueError("Could not find in_channels in the model architecture.")

    hidden_units_match = re.search(r'GRU\((\d+)', model_architecture)
    if hidden_units_match:
        hidden_units = int(hidden_units_match.group(1))
    else:
        raise ValueError("Could not find hidden_units in the model architecture.")

    dropout_match = re.search(r'Dropout\(p=([\d.]+)', model_architecture)
    if dropout_match:
        dropout = float(dropout_match.group(1))
    else:
        raise ValueError("Could not find dropout in the model architecture.")

    # Other architecture details (e.g., out_features)
    out_features_match = re.search(r'out_features=(\d+)', model_architecture)
    if out_features_match:
        out_features = int(out_features_match.group(1))
    else:
        raise ValueError("Could not find out_features in the model architecture.")







    #num_classes = model_info['num_classes']
    #in_channels = model_info['in_channels']
    #hidden_units = model_info['hidden_units']
    #dropout = model_info['dropout']

    model = model_builder.LiteGRUv2(num_classes=NUM_CLASSES,
                                    in_channels=in_channels,
                                    hidden_units=hidden_units,
                                    dropout=dropout).to(torch.device('cpu'))

    # Extract other relevant information from model_info if needed

    return model




if args.model_option == "new":
    model = model_builder.LiteGRUv2(num_classes=NUM_CLASSES,
                                    in_channels=N_MFCCS-1,
                                    hidden_units=HIDDEN_UNITS,
                                    dropout=DROPOUT).to(torch.device('cpu'))

    print(f"[INFO] NEW model active: {type(model).__name__}\n")

else:
    # Train an old model
    if args.model_option == "old":
        if args.old_model_path is None:
            parser.error("--old_model_path argument is required when using the old model option")
            sys.exit(1)

        # Create an instance of the old model
        model = load_model_with_params("/content/drive/MyDrive/pytorch/LiteGRUv1/cv-corpus-14.0-delta-2023-06-23/en/models/"+args.old_model_path)

        print(f"[INFO] OLD model active: {type(model).__name__}\n")


# Apply weight initialization (e.g., Xavier/Glorot initialization)
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)


# Set loss and optimizer
loss_fn = torch.nn.CTCLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

print(f"[INFO] loss_fn active: {loss_fn}")

print(f"[INFO] optimizer active: {optimizer}\n")

print(f"[INFO] about to train...\n")

# Start training with help from engine.py
log_book = engine.train(model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        loss_fn=loss_fn,
                        optimizer=optimizer,
                        epochs=NUM_EPOCHS,
                        num_classes=NUM_CLASSES,
                        device=device,
                        )

print("[INFO] did the train\n")

print(f"log_book: {log_book}\n")

# Save the model with help from utils.py

# The base filename you want to use
base_filename = "LiteGRUv2_model_test.pth"

# The directory where you want to save the models
target_dir = "/content/drive/MyDrive/pytorch/LiteGRUv1/cv-corpus-14.0-delta-2023-06-23/en/models"


# Check if the base filename exists
if os.path.exists(os.path.join(target_dir, base_filename)):
    # If the base filename exists, find the next available number
    # Loop until you find a unique filename
    file_number = 1
    while True:
        new_filename = f"{base_filename[:-4]}_{file_number}.pth"
        if not os.path.exists(os.path.join(target_dir, new_filename)):
            break
        file_number += 1
else:
    # If the base filename doesn't exist, use it directly
    new_filename = base_filename

# Save the model with the new filename
utils.save_model(model=model,
                 target_dir=target_dir,
                 model_name=new_filename
                 )

dict_save_path = os.path.join(target_dir, f"{new_filename[:-4]}.pkl")  # Change extension to .pkl

# Save the Log book using pickle
with open(dict_save_path, 'wb') as f:
    pickle.dump(log_book, f)

print(f"[INFO] log_book has been pickled as {new_filename[:-4]}.pkl \n")



Writing train.py


In [13]:
#%%writefile main_prompt.py
import multiprocessing

import random
import numpy as np
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

import os
import subprocess
import ipywidgets as widgets
from IPython.display import display, clear_output

if __name__ == '__main__':
    OLD_MODEL = "LiteGRUv2_model_test_12.pth"

    # Default values
    def_epochs = 5
    def_batch = 64
    def_hidden = 512
    def_learn = 0.001
    options = ["old", "new"]
    def_model = "new"
    def_n_mfccs = 13
    def_dropout = 0.1

    # Create the input widgets
    input_variable_1 = widgets.Text(description='Number of epochs:', value=str(def_epochs), style={'description_width': 'initial'}, align_self='center')
    input_variable_2 = widgets.Text(description='Batch size:', value=str(def_batch), style={'description_width': 'initial'}, align_self='center')
    input_variable_3 = widgets.Text(description='Hidden units in layer:', value=str(def_hidden), style={'description_width': 'initial'}, align_self='center')
    input_variable_4 = widgets.Text(description='Learning rate:', value=str(def_learn), style={'description_width': 'initial'}, align_self='center')
    input_variable_5 = widgets.RadioButtons(description='Train "old" or "new" model?', options=options, value=def_model, style={'description_width': 'initial'}, align_self='center')
    input_variable_6 = widgets.Text(description='Name of old model:', value=OLD_MODEL, style={'description_width': 'initial'}, align_self='center')
    input_variable_7 = widgets.IntText(description='Number of MFCC coefficients:', value=def_n_mfccs, style={'description_width': 'initial'}, align_self='center')
    input_variable_8 = widgets.FloatText(description='Dropout rate:', value=def_dropout, style={'description_width': 'initial'}, align_self='center')

    # Create a button widget
    run_button = widgets.Button(description='Run Script')
    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            print(f"Button clicked, script running...")

            clear_output()  # Clear the output area before running the script
            variable_1 = input_variable_1.value or def_epochs
            variable_2 = input_variable_2.value or def_batch
            variable_3 = input_variable_3.value or def_hidden
            variable_4 = input_variable_4.value or def_learn
            variable_5 = input_variable_5.value or def_model
            variable_6 = input_variable_6.value or OLD_MODEL
            variable_7 = input_variable_7.value or def_n_mfccs
            variable_8 = input_variable_8.value or def_dropout

            # Construct the command to run the .py script using the entered variables IN ORDER ARE:
            command = f'python train.py --num_epochs {variable_1} --batch_size {variable_2} --hidden_units {variable_3} --learning_rate {variable_4}'

            command += f' --model_option {variable_5}'

            command += f' --old_model_path {variable_6}'

            command += f' --n_mfccs {variable_7} --dropout {variable_8}'

            # Run the command in a subprocess with real-time output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            for line in process.stdout:
                print(line, end='')

    run_button.on_click(on_button_clicked)

    # Create a VBox layout for the input fields and button
    input_layout = widgets.VBox([input_variable_1, input_variable_2, input_variable_3, input_variable_4, input_variable_5, input_variable_6, run_button])

    # Display the form
    display(input_layout, output)


Output()

In [11]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
!cat /proc/cpuinfo

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
%%bash
cat /proc/cpuinfo

Couldn't find program: 'bash'


 Volume in drive C has no label.
 Volume Serial Number is 4C8C-72CB

 Directory of C:\Users\Mr Cab Driver\Documents\aCreativeHub\Projekti\Speech-to-Text\LiteGRUv1\anaconda\LiteGRUv2_v2_7

16.08.2023.  00:37    <DIR>          .
16.08.2023.  00:37    <DIR>          ..
16.08.2023.  00:35               589 char_to_idx.py
15.08.2023.  22:43             2.084 combine_and_plot.py
16.08.2023.  00:35             8.688 data_setup.py
16.08.2023.  00:35            25.881 engine.py
16.08.2023.  00:37            82.789 LiteGRUv2_v2.7_transfer_from_colab.ipynb
15.08.2023.  23:31            14.014 LiteGRUv2_v2_7.7z
15.08.2023.  23:20             3.460 main_prompt.py
16.08.2023.  00:35             2.242 model_builder.py
16.08.2023.  00:35             3.447 plot_loss_curves.py
16.08.2023.  00:35             4.747 prediction.py
16.08.2023.  00:35            10.575 train.py
16.08.2023.  00:35             3.086 utils.py
15.08.2023.  23:49    <DIR>          __pycache__
              12 File(s)        161.60